In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tabulate import tabulate

# Read data from excel 
data = pd.read_excel('../preprocessing/Dry_Bean_Dataset.xlsx')

X = data.drop(columns = ['Class'])
y = data['Class']

# label_encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)


In [6]:
# split the training data and testing data, then do standarization
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42, shuffle=True)
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# construct the multi-layer perceptron model
# alpha: regularization parameter, solver: optimization algorithm, tol: stop condition
mlp_clf = MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=100, alpha = 1e-4,
                        solver='lbfgs', tol=1e-6, random_state=42)

mlp_clf.fit(X_train_scaled, y_train)

y_train_pred = mlp_clf.predict(X_train_scaled)
y_test_pred = mlp_clf.predict(X_test_scaled)

c:\Users\14198\anaconda3\envs\Yingwen\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [8]:
accuracy_train = accuracy_score(y_train_pred, y_train)
accuracy_test = accuracy_score(y_test_pred, y_test)

print(f"Training accuracy: {accuracy_train:.4f}")
print(f"Testing accuracy: {accuracy_test:.4f}")


model_evaluation_metrics1 = {}

conf_matrix_train = confusion_matrix(y_train, y_train_pred)
classif_report_train = classification_report(y_train, y_train_pred)

model_evaluation_metrics1 = {
    'Confusion Matrix': conf_matrix_train,
    'Classification Report': classif_report_train
}

print(f"MLP Evaluation Metrics:")
print("Confusion Matrix:")
print(tabulate(conf_matrix_train, headers=mlp_clf.classes_, tablefmt='grid'))
print("\nClassification Report:")
print(classif_report_train)

model_evaluation_metrics2 = {}

conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classif_report_test = classification_report(y_test, y_test_pred)

model_evaluation_metrics2 = {
    'Confusion Matrix': conf_matrix_test,
    'Classification Report': classif_report_test
}

print(f"MLP Evaluation Metrics:")
print("Confusion Matrix:")
print(tabulate(conf_matrix_test, headers=mlp_clf.classes_, tablefmt='grid'))
print("\nClassification Report:")
print(classif_report_test)

Training accuracy: 0.9407
Testing accuracy: 0.9324
MLP Evaluation Metrics:
Confusion Matrix:
+-----+-----+------+------+------+------+------+
|   0 |   1 |    2 |    3 |    4 |    5 |    6 |
+=====+=====+======+======+======+======+======+
| 875 |   0 |   28 |    0 |    5 |    5 |   14 |
+-----+-----+------+------+------+------+------+
|   0 | 361 |    0 |    0 |    0 |    0 |    0 |
+-----+-----+------+------+------+------+------+
|  17 |   0 | 1112 |    0 |   12 |    1 |    9 |
+-----+-----+------+------+------+------+------+
|   0 |   0 |    0 | 2351 |    2 |   25 |  125 |
+-----+-----+------+------+------+------+------+
|   2 |   0 |   18 |   13 | 1283 |    0 |   24 |
+-----+-----+------+------+------+------+------+
|   3 |   0 |    0 |   25 |    0 | 1354 |   26 |
+-----+-----+------+------+------+------+------+
|   6 |   0 |    2 |  161 |   21 |   21 | 1626 |
+-----+-----+------+------+------+------+------+

Classification Report:
              precision    recall  f1-score   supp

接下来可以尝试
1. 增加1，5，6类（错误率较高）的数据
2. 交叉验证调参

In [9]:
# First try cross-validation 
from sklearn.model_selection import GridSearchCV, StratifiedKFold

mlp_clf2 = MLPClassifier(early_stopping=True, random_state=42)

param_grid = {
    'hidden_layer_sizes':[(50,), (50, 20), (100,)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'lbfgs'],
    'alpha': [0.0001, 0.001],
    'max_iter': [200, 500],
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(
    estimator=mlp_clf2,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy',  # 可根据需求改为 'f1_macro' 等
    n_jobs=-1,           # 并行计算
    verbose=1            # 打印进度
)

# 在标准化后的训练集上搜索
grid.fit(X_train_scaled, y_train)

# 输出最佳参数和交叉验证分数
print("Best parameters:", grid.best_params_)
print("Best CV accuracy:", grid.best_score_)

# 在测试集上评估
best_mlp = grid.best_estimator_
test_score = best_mlp.score(X_test_scaled, y_test)
print("Test set accuracy:", test_score)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: {'activation': 'relu', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'max_iter': 200, 'solver': 'lbfgs'}
Best CV accuracy: 0.9278893286293043
Test set accuracy: 0.9297257590597453


c:\Users\14198\anaconda3\envs\Yingwen\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [10]:
y_test_pred = best_mlp.predict(X_test_scaled)

model_evaluation_metrics3 = {}

conf_matrix_test = confusion_matrix(y_test, y_test_pred)
classif_report_test = classification_report(y_test, y_test_pred)

model_evaluation_metrics3 = {
    'Confusion Matrix': conf_matrix_test,
    'Classification Report': classif_report_test
}

print(f"MLP Evaluation Metrics:")
print("Confusion Matrix:")
print(tabulate(conf_matrix_test, headers=mlp_clf.classes_, tablefmt='grid'))
print("\nClassification Report:")
print(classif_report_test)

MLP Evaluation Metrics:
Confusion Matrix:
+-----+-----+-----+-----+-----+-----+-----+
|   0 |   1 |   2 |   3 |   4 |   5 |   6 |
+=====+=====+=====+=====+=====+=====+=====+
| 364 |   2 |  17 |   0 |   2 |   3 |   7 |
+-----+-----+-----+-----+-----+-----+-----+
|   0 | 161 |   0 |   0 |   0 |   0 |   0 |
+-----+-----+-----+-----+-----+-----+-----+
|  16 |   0 | 451 |   0 |   5 |   2 |   5 |
+-----+-----+-----+-----+-----+-----+-----+
|   0 |   0 |   0 | 972 |   0 |  10 |  61 |
+-----+-----+-----+-----+-----+-----+-----+
|   2 |   0 |   7 |   7 | 557 |   0 |  15 |
+-----+-----+-----+-----+-----+-----+-----+
|   5 |   0 |   0 |  12 |   0 | 590 |  12 |
+-----+-----+-----+-----+-----+-----+-----+
|   3 |   0 |   1 |  73 |   8 |  12 | 702 |
+-----+-----+-----+-----+-----+-----+-----+

Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.92      0.93       395
           1       0.99      1.00      0.99       161
           2       0.95